<a href="https://colab.research.google.com/github/YusairahYamany/TheEyeOfTheArtist/blob/main/TheEyeOfTheArtist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount your Google Drive into Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

## Change this code to the path of your dataset in Google Drive
dataset_directory = '/content/drive/My Drive/ArtDataset'
dataset = os.listdir(dataset_directory)

for item in dataset:
  num_files = len(os.listdir(f"{dataset_directory}/{item}"))
  print(f"{item}: {num_files} files")

Watercolour: 100 files
Oil: 100 files


In [ ]:
import tensorflow as tf

In [ ]:
num_classes = len(os.listdir(dataset_directory))

if num_classes == 2:
  n = 1
else:
  n = num_classes

prediction_layer = tf.keras.layers.Dense(n)

In [ ]:
test_set_size = 0.2

BATCH_SIZE = 32
IMG_SIZE = (224, 224)


train_dataset = tf.keras.utils.image_dataset_from_directory(dataset_directory,
                                                            validation_split=test_set_size,
                                                            subset="training",
                                                            seed=123,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

test_dataset = tf.keras.utils.image_dataset_from_directory(dataset_directory,
                                                           validation_split=test_set_size,
                                                           subset="training",
                                                           seed=123,
                                                           batch_size=BATCH_SIZE,
                                                           image_size=IMG_SIZE)

Found 175 files belonging to 2 classes.
Using 140 files for training.
Found 175 files belonging to 2 classes.
Using 140 files for training.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

preprocess_input = tf.keras.applications.resnet50.preprocess_input

IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')])

initial_epochs = 100
history = model.fit(train_dataset,
                    epochs=initial_epochs)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
5/5 [==============================] - 39s 3s/step - loss: 0.9052 - accuracy: 0.5000
Epoch 2/100
5/5 [==============================] - 18s 3s/step - loss: 0.7596 - accuracy: 0.5714
Epoch 3/100
5/5 [==============================] - 19s 3s/step - loss: 0.7786 - accuracy: 0.5786
Epoch 4/100
5/5 [==============================] - 18s 3s/step - loss: 0.8614 - accuracy: 0.5286
Epoch 5/100
5/5 [==============================] - 18s 3s/step - loss: 0.8170 - accuracy: 0.5286
Epoch 6/100
5/5 [==============================] - 18s 3s/step - loss: 0.7102 - accuracy: 0.5857
Epoch 7/100
5/5 [==============================] - 19s 3s/step - loss: 0.6748 - accuracy: 0.6429
Epoch 8/100
5/5 [==============================] - 18s 3s/step - loss: 0.6329 - accuracy: 0.6286
Epoch 9/100
5/5 [==============================] - 18s 3s/step - loss: 0.7054 - accuracy: 0.5857
Epoch 10/100
5/5 [==============================] - 17s 3s/ste

In [ ]:
import numpy as np

y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_true

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 0], dtype=int32)

In [ ]:
predictions = model.predict(test_dataset)

predictions = (predictions > 0).astype(int).squeeze()

predictions

5/5 [==============================] - 18s 3s/step


array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1])

In [ ]:
def calculate_accuracy(y_true, predictions):

  accuracy = np.sum(y_true == predictions)/len(y_true)

  return accuracy

calculate_accuracy(y_true, predictions)

0.4785714285714286

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, predictions)

0.4785714285714286

In [ ]:
import cv2
import numpy as np

def predict_artwork(image_path, model):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))  # Resize to match model input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = tf.keras.applications.resnet50.preprocess_input(image)

    # Predict the class probabilities
    probabilities = model.predict(image)[0]

    # Get the class with the highest probability
    predicted_class = np.argmax(probabilities)

    # Assuming you have a list of class names, replace class_names with your actual class names
    class_names = ['Watercolour', 'Oil']  # Replace with your actual class names

    predicted_class_name = class_names[predicted_class]
    confidence = probabilities[predicted_class]

    return predicted_class_name, confidence

# Assuming 'model' is your trained model
# Replace 'path_to_image' with the actual path of the artwork image
path_to_image = '/content/w27.jpeg'
predicted_class, confidence = predict_artwork(path_to_image, model)

print(f'Predicted Class: {predicted_class}')
print(f'Confidence: {confidence}')


1/1 [==============================] - 1s 844ms/step
Predicted Class: Watercolour
Confidence: 0.5493518710136414
